In [1]:
%autosave 0
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Téléchargez la liste de stop words en français
nltk.download('stopwords')

# Importez la liste de stop words en français
from nltk.corpus import stopwords
stop_words_french = set(stopwords.words('french'))

Autosave disabled


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_test = pd.read_excel("df_clean.xlsx")
df_test.head(10)

,Client,Commentaire,Note,Sentiment
0,marc lesterpt,Je commande mes pneus de motos et de voitures ...,5,Positif
1,Séverine M,"BonjourLe site est génial, très bien expliqué ...",5,Positif
2,PINTO NUNO,"Tout était parfait, timing parfait dans les da...",5,Positif
3,client,Pneus d'un tres bon rapport qualité prix. Les ...,5,Positif
4,Vignaux,"Bon suivi de la commande,les questions sont im...",4,Positif
5,cliente juliema,Ce service sur place est d un grand secours. R...,5,Positif
6,cliente,La commande sur internet c'est très bien passé...,5,Positif
7,Manu,"RAS, le rapport qualité prix est très correct....",5,Positif
8,Xavier,Fidèle depuis de nombreuses années j aï toujou...,5,Positif
9,Sonia Nutrition,M'a été recommandé. Meilleur rapport qualité p...,5,Positif


In [3]:
# (a) Supprimer les caractères spéciaux, chiffres, valeurs manquantes et mettre le texte en minuscules.
df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
df_test['Commentaire'] = df_test['Commentaire'].fillna('') 
df_test.head(10)

C:\Users\micka\AppData\Local\Temp\ipykernel_1276\550896256.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test['Commentaire'] = df_test['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()


,Client,Commentaire,Note,Sentiment
0,marc lesterpt,je commande mes pneus de motos et de voitures ...,5,Positif
1,Séverine M,bonjourle site est gnial trs bien expliqu pour...,5,Positif
2,PINTO NUNO,tout tait parfait timing parfait dans les date...,5,Positif
3,client,pneus dun tres bon rapport qualit prix les mei...,5,Positif
4,Vignaux,bon suivi de la commandeles questions sont imm...,4,Positif
5,cliente juliema,ce service sur place est d un grand secours ri...,5,Positif
6,cliente,la commande sur internet cest trs bien passe r...,5,Positif
7,Manu,ras le rapport qualit prix est trs correct tou...,5,Positif
8,Xavier,fidle depuis de nombreuses annes j a toujours ...,5,Positif
9,Sonia Nutrition,ma t recommand meilleur rapport qualit prix et...,5,Positif


In [4]:
# (b) Division du DataFrame en ensembles d'entraînement et de test.
X_train, X_test, y_train, y_test = train_test_split(df_test['Commentaire'], df_test['Sentiment'], test_size=0.2, random_state=42)

In [5]:
# (c) Vectorisation du texte avec CountVectorizer.
vectorizer = TfidfVectorizer(stop_words=list(stop_words_french))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [6]:
# (d) Création du classificateur GradientBoostingClassifier.
clf = make_pipeline(StandardScaler(with_mean=False), RandomForestClassifier(random_state=42))

In [7]:
# (e) Entraînement du classificateur sur l'ensemble d'entraînement.
clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)

In [8]:
# (f) Affichage du rapport de classification du modèle Gradient Boosting.
print("Rapport de classification :\n", classification_report(y_test, y_pred))

Rapport de classification :
               precision    recall  f1-score   support

      Neutre       0.51      0.01      0.02      2532
     Négatif       0.80      0.74      0.77      7296
     Positif       0.91      0.99      0.95     36649

    accuracy                           0.89     46477
   macro avg       0.74      0.58      0.58     46477
weighted avg       0.87      0.89      0.87     46477



In [9]:
# (g) Calcul et affichage de la matrice de confusion.
confusion_matrix = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Classe prédite,Neutre,Négatif,Positif
Classe réelle,,,
Neutre,25,791,1716
Négatif,15,5383,1898
Positif,9,536,36104
